
# Regression mit Train-Test-Split

Dieses Notebook erweitert das einfache Beispiel: Wir prüfen, ob unser Modell auch auf neuen Daten funktioniert. Dazu teilen wir die Daten in Trainings- und Testset, trainieren eine lineare Regression und evaluieren sie mit mehreren Kennzahlen.



Ziele:

1. Aufteilen der Daten in `train` und `test`.
2. Modell nur auf `train` fitten.
3. Leistung auf `test` messen.
4. Optional: Cross-Validation für stabilere Kennzahlen.


In [13]:

import pandas as pd
import altair as alt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, KFold, cross_validate



## 1. Daten laden

Wie zuvor nutzen wir `data/data-ads.csv` mit Werbebudget (`ads`) und Verkäufen (`sales`).


In [14]:

df = pd.read_csv("data/data-ads-2.csv")
df.head()


,sales,ads
0,8300,300
1,5900,550
2,4400,600
3,3100,650
4,5700,700



## 2. Train-Test-Split

Wir reservieren 20 % der Daten für den abschließenden Test. `random_state` macht das Ergebnis reproduzierbar.


In [15]:

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

print(f"Trainingsdaten: {train_df.shape[0]} Zeilen")
print(f"Testdaten:      {test_df.shape[0]} Zeilen")


Trainingsdaten: 80 Zeilen
Testdaten:      20 Zeilen



Ein kurzer Überblick über das Trainingsset stellt sicher, dass die Verteilung plausibel aussieht.


In [16]:

train_df.describe().T


,count,mean,std,min,25%,50%,75%,max
sales,80.0,38031.25,20375.404593,3100.0,20225.0,39300.0,52125.0,76500.0
ads,80.0,7506.25,4387.859942,550.0,3587.5,7900.0,10737.5,14950.0



Streudiagramm ausschließlich auf Basis der Trainingsdaten:


In [ ]:

train_scatter = (
    alt.Chart(train_df)
    .mark_circle(size=80, color="#1f77b4")
    .encode(
        x=alt.X("ads", title="Werbebudget"), 
        y=alt.Y("sales", title="Verkäufe"))
    .properties(title="Training: Werbung vs. Verkäufe")
)
train_scatter


alt.Chart(...)


## 3. Modell trainieren

Wir definieren Features und Zielvariable getrennt für Train und Test.


In [18]:

feature = "ads"
target = "sales"

X_train = train_df[[feature]]
y_train = train_df[target]
X_test = test_df[[feature]]
y_test = test_df[target]



Das Modell wird ausschließlich auf den Trainingsdaten gefittet.


In [19]:

model = LinearRegression()
model.fit(X_train, y_train)


,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False



Zur Einordnung werfen wir wieder einen Blick auf die Parameter.


In [20]:

coef_table = pd.DataFrame(
    {
        "Parameter": ["Achsenabschnitt", "Steigung"],
        "Wert": [model.intercept_, model.coef_[0]],
    }
)
coef_table


,Parameter,Wert
0,Achsenabschnitt,3449.523611
1,Steigung,4.607058



## 4. Evaluate

Wir vergleichen Trainings- und Testergebnisse. Große Unterschiede deuten auf Overfitting hin.


In [21]:

def evaluate_split(name, X, y):
    preds = model.predict(X)
    mse = mean_squared_error(y, preds)
    return {
        "Datensatz": name,
        "MSE": mse,
        "RMSE": mse ** 0.5,
        "R^2": r2_score(y, preds),
    }

results = [
    evaluate_split("Train", X_train, y_train),
    evaluate_split("Test", X_test, y_test),
]

pd.DataFrame(results).round(3)


,Datensatz,MSE,RMSE,R^2
0,Train,6424564.225,2534.672,0.984
1,Test,3413887.078,1847.671,0.991



Eine Grafik macht deutlich, wie gut die Vorhersagen die Testpunkte treffen.


In [22]:

test_with_pred = test_df.assign(prediction=model.predict(X_test))
test_sorted = test_with_pred.sort_values(feature)

points = (
    alt.Chart(test_with_pred)
    .mark_circle(size=80, color="#1f77b4")
    .encode(x=alt.X("ads", title="Werbebudget"), y=alt.Y("sales", title="Reale Verkäufe"))
)
line = (
    alt.Chart(test_sorted)
    .mark_line(color="#d62728", size=3)
    .encode(x="ads", y="prediction")
)
(points + line).properties(title="Testdaten vs. Vorhersage")


alt.LayerChart(...)


## 5. Bonus: Cross-Validation

Statt eines einzigen Splits können wir mehrere Folds verwenden. Das stabilisiert die Kennzahlen und gibt eine realistischere Spanne.


In [23]:

cv = KFold(n_splits=5, shuffle=True, random_state=42)
cv_results = cross_validate(
    LinearRegression(),
    X_train,
    y_train,
    cv=cv,
    scoring=("r2", "neg_mean_squared_error"),
)
summary = pd.DataFrame(
    {
        "Fold": range(1, cv_results["test_r2"].shape[0] + 1),
        "R^2": cv_results["test_r2"],
        "RMSE": (-cv_results["test_neg_mean_squared_error"]) ** 0.5,
    }
)
summary.round(3)


,Fold,R^2,RMSE
0,1,0.992,1964.579
1,2,0.980,2878.861
2,3,0.982,2298.399
3,4,0.973,3311.229
4,5,0.980,2703.359



Zum Abschluss eine kurze Interpretation:

- Liegen Train- und Test-RMSE dicht beieinander, verallgemeinert das Modell gut.
- Cross-Validation bestätigt die Schätzungen und zeigt deren Streuung.
- Schon dieses einfache Beispiel reicht, um typische Schritte jedes Regressionsprojekts zu verstehen.
